# Toy Experiment for Paper "When Scores Learn Geometry"

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define model
class MLP(nn.Module):
    def __init__(self, x_dim, t_dim, hidden_dim=128, num_layers=3):
        in_dim = x_dim + t_dim
        out_dim = x_dim
        super(MLP, self).__init__()
        layers = [nn.Linear(in_dim, hidden_dim), nn.ReLU()]
        for _ in range(num_layers - 2):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_dim, out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, xt, t):
        return self.net(torch.cat([xt, t.unsqueeze(-1)], dim=-1))

/home/chaoyipan/miniforge3/envs/sam3d-objects/lib/python3.11/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Generate data from $x_0^2 + x_1^2 = 1$ manifold


In [2]:
num_train_samples = 1000
batch_size = 128

theta = torch.linspace(0, 2 * np.pi, num_train_samples, device=device)
x = torch.stack([torch.cos(theta), torch.sin(theta)], dim=-1)

dataset = TensorDataset(x)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

/home/chaoyipan/miniforge3/envs/sam3d-objects/lib/python3.11/site-packages/torch/cuda/__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5090 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


train velocity field

In [ ]:
num_epochs = 100
model = MLP(x_dim=2, t_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
    epoch_loss_sum = 0.0
    for x1, in dataloader:  # Unpack Tuple for x1
        t = torch.rand((batch_size, ), device=device)
        x0 = torch.randn((batch_size, 2), device=device)
        xt = t * x1 + (1 - t) * x0
        vel_target = (x1 - x0) / 1.0
        vel_pred = model(xt, t)
        loss = F.mse_loss(vel_pred, vel_target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss_sum += loss.item()
    epoch_loss = epoch_loss_sum / len(dataloader)
    tqdm.write(f"Epoch {epoch} Loss: {epoch_loss:.4f}")


NameError: name 'MLP' is not defined